Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [23]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt



#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
from sklearn.model_selection import train_test_split

yf.pdr_override()

In [24]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol= 'STC'


In [37]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

#end=dt.datetime.now()
#start=dt.datetime(2020,11,8)
#df=pdr.get_data_yahoo(stock_symbol,start,end)
end=dt.datetime.now()
start=dt.datetime(2020,11,8)
df=pdr.get_data_yahoo(stock_symbol,start,end)
df=df.iloc[:,:4]
#df = pdr.get_data_yahoo(stock_symbol, start_date= dt.datetime(2020,11,8),end_date= dt.datetime.now())
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2020-11-09,45.860001,46.180000,40.950001,41.049999
2020-11-10,41.419998,43.169998,41.020000,42.650002
2020-11-11,42.680000,42.680000,41.369999,42.380001
2020-11-12,42.169998,42.810001,41.500000,42.570000
2020-11-13,42.340000,43.000000,42.110001,42.689999
...,...,...,...,...
2021-11-02,71.860001,72.620003,71.180000,71.459999
2021-11-03,71.430000,73.180000,71.389999,72.180000
2021-11-04,72.330002,72.980003,71.050003,71.900002


In [26]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
df['Open_1_Change'] = df['Open']-df['Open'].shift(1)
df = df.dropna()
X = df
X

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2020-11-30,43.380001,43.619999,41.840000,41.869999,43.413334,-0.119999
2020-12-01,42.330002,43.490002,42.330002,42.770000,43.528000,-1.049999
2020-12-02,42.779999,43.470001,42.639999,43.169998,43.562667,0.449997
2020-12-03,43.169998,44.439999,42.849998,44.320000,43.692000,0.389999
2020-12-04,44.500000,46.720001,44.400002,46.480000,43.952667,1.330002
...,...,...,...,...,...,...
2021-11-02,71.860001,72.620003,71.180000,71.459999,70.367333,0.110001
2021-11-03,71.430000,73.180000,71.389999,72.180000,70.647333,-0.430000
2021-11-04,72.330002,72.980003,71.050003,71.900002,70.853334,0.900002


In [27]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where(df['Close'].shift(-1) - df['Close'] < 0, -1, 1)
y

array([ 1,  1,  1,  1, -1,  1,  1,  1,  1, -1,  1, -1, -1,  1, -1, -1, -1,
        1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1,  1,
       -1, -1, -1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,
        1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1,  1,  1,  1,
        1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1,  1,  1,
        1,  1,  1,  1, -1, -1, -1,  1, -1, -1, -1,  1,  1,  1,  1, -1,  1,
        1, -1,  1, -1,  1,  1,  1, -1, -1, -1,  1,  1, -1, -1,  1,  1, -1,
       -1, -1,  1, -1,  1,  1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1,  1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1, -1,  1,  1,
       -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1,
        1, -1,  1,  1,  1,  1, -1,  1,  1, -1, -1,  1,  1,  1, -1,  1,  1,
       -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1,  1, -1,  1,  1,
        1,  1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1, -1,
        1,  1,  1,  1, -1

In [28]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.


index = int(0.7*len(X))
X_train, X_test, y_train , y_test =train_test_split(X, y, test_size= index)
#X_train, X_test, y_train , y_test


In [29]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)


In [30]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[-0.23225028405267212]
1,High,[-0.3772472124737116]
2,Low,[0.022092512432201172]
3,Close,[0.23618352937081286]
4,Close_15_Rolling,[0.4080934395650123]
5,Open_1_Change,[0.23131129052591537]


In [31]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)


In [32]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

metrics.confusion_matrix(y_test,predictions)


array([[27, 45],
       [34, 60]], dtype=int64)

In [50]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

logistic.score(X_test, y_test)



0.5240963855421686

In [55]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

from sklearn.model_selection import cross_val_score
cross_val =cross_val_score(logistic,X_test,y_test)


C:\Users\bsma\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\bsma\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 